In [ ]:
import numpy as np
from astropy.io import fits
import struct
import numpy as np
from glob import glob

import matplotlib.pyplot as plt

%matplotlib inline

def raw_to_fits(raw_file, shape=(1080,1440), destination=None, ADC_bits = 12, simu_artifacts = False):
    """
    Converts a .raw file to a FITS formatted image
    raw_file : The file path of the raw image
    shape : A list or tuple with (# of rows, # of columns) for the image
    destination : an alternate destination/filename for the FITS image
       (default is to replace the .raw extension and save in the same folder)
    """
    tint = float(raw_file.split("us.raw")[0].split("_")[-1])/1e6

    # Get shape of image
    nrows, ncols = shape
    # Open raw file
    with open(raw_file, 'rb') as raw:
        raw_img = raw.read()
    # Check file size / format
    filesize = len(raw_img)
    if filesize == nrows*ncols: # 8-bit
        size = 'B'
        file_bits = 8
    elif filesize == nrows*ncols*2: # 16-bit
        size = 'H'
        file_bits = 16
    else: # No size match
        raise ValueError("The image shape provided does not match the length of the file")
    # Format binary string
    fmt = f"<{nrows*ncols}{size}"
    # Convert byte string to numpy array
    byte_array = np.array(struct.unpack(fmt, raw_img)).reshape(nrows, ncols)
    if file_bits > ADC_bits:
        byte_array = byte_array/2**(file_bits-ADC_bits)

    if simu_artifacts:
        ny,nx = byte_array.shape

        # Adding dark
        dark_current = 1.25 #ADU/s
        byte_array += dark_current*tint

        # multiplying flat
        x_grid,y_grid = np.meshgrid(np.arange(nx),np.arange(ny))
        flat_field = np.exp(-0.5*((x_grid - nx//2)**2/(3*nx)**2+(y_grid - ny//2)**2/(3*ny)**2))
        byte_array *= flat_field

        state = np.random.get_state()
        np.random.seed(0)

        # adding hot pixels
        hot_pix_indices = np.random.randint(0, high=nx*ny-1, size=int(0.01*nx*ny))
        hot_pix_indices = np.unravel_index(hot_pix_indices, (ny,nx))
        byte_array[hot_pix_indices] *= tint*np.clip(30*np.random.randn(int(0.01*nx*ny)),dark_current,2**ADC_bits)

        # adding cold pixels
        cold_pix_indices = np.random.randint(0, high=nx*ny-1, size=int(0.01*nx*ny))
        cold_pix_indices = np.unravel_index(cold_pix_indices, (ny,nx))
        byte_array[cold_pix_indices] *= np.random.uniform(0,0.5,size=int(0.01*nx*ny))

        np.random.set_state(state)

        # Adding bias
        byte_array += 64

        # adding readnoise
        readnoise = 3 #ADU
        byte_array += readnoise*np.random.randn(ny,nx)

        byte_array = np.clip(byte_array,0,2**ADC_bits)

    # FITS file name
    if destination is None:
        fits_file = raw_file.replace('.raw', '.fits')
    else:
        fits_file = destination
    if simu_artifacts:
        fits_file = fits_file.replace(".fits", "_simu.fits")
    # Save as FITS
    hdu = fits.PrimaryHDU(byte_array)
    hdu.writeto(fits_file,overwrite=True)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Press the green button in the gutter to run the script.
mydir = "/content/drive/Shareddrives/AY120_2024/Student Data Files/Lab #1 Data Files/Lab #1 Group 6A/lab1jupiter_10000us.raw"

print(mydir)
for filename in glob(mydir):
    if len(glob(filename.replace(".raw",".fits"))) >= 1:
        continue
    print(filename)
    # try both (width, height) and (height, width) for the shape keyword in case the image is funky with stripes.
   # raw_to_fits(filename, shape=(1536,2048))
    #raw_to_fits(filename, shape=(1080,1440))
    raw_to_fits(filename, shape=(1240,1616))

/content/drive/Shareddrives/AY120_2024/Student Data Files/Lab #1 Data Files/Lab #1 Group 6A/lab1jupiter_10000us.raw
/content/drive/Shareddrives/AY120_2024/Student Data Files/Lab #1 Data Files/Lab #1 Group 6A/lab1jupiter_10000us.raw
